In [307]:
# dI (t) = −α_1*I(t) − α_2*I(t)^2 + (α_3 * I(t) + q) * I (t)^(1−(1/n))
using SymPy
using Plots
using DifferentialEquations
@vars x

f(x)=2*sin(x)+3*x^(1/2)
u0 = f(1)
println("u0=",u0)

p1 = plot(f, grid=false, 1,30)

df = diff(f)

#df(x) = df(x)+f(x)-f(x) = f(x)+g(x)

g(x) = df(x)-f(x)
println("f(x)=",f(x))
println("df(x)=",df(x))
println("g(x)=",g(x))
println(df(x)==f(x)+g(x))

F(u,p,t) = u + 1.5*t^(-0.5) - 3*t^0.5 - 2*sin(t) + 2*cos(t)


tspan = (1.0,15.0)
prob = ODEProblem(F,u0,tspan)

sol = DF.solve(prob)

graph = plot(f, tspan[1], tspan[2], color="blue")
plot!(graph, sol, ls=:dash);

u0=4.6829419696157935
f(x)=3*x^0.5 + 2*sin(x)
df(x)=1.5*x^(-0.5) + 2*cos(x)
g(x)=1.5*x^(-0.5) - 3*x^0.5 - 2*sin(x) + 2*cos(x)
true


In [313]:
using SymPy
using Plots
using DifferentialEquations
using CSV
using StatsPlots
using StatsBase


function find_fit(values, equa_diff, func, variable, params_span, step=10, n_adns=100)
    adns = create_adns(params_span, n_adns)
    n_mutants = floor(Int, n_adns*0.15)
    n_childs = floor(Int, n_adns*0.15)
    n_parents = floor(Int, (n_adns-n_mutants-n_childs)*0.3)
    
    for i in 1:step
        scores = get_scores(adns, values, equa_diff, func, variable)
        sort!(scores, by=el->el[2])
        println("best = ", scores[end])
        
        scores = scores[n_mutants+n_childs:end]
        adns = [score[1] for score in scores]
        
        new_mutants = create_adns(params_span, n_mutants)
        new_childs = create_childs(adns, n_childs, n_parents)
        adns = vcat(new_mutants, new_childs, adns)
    end
    
    return scores[end]
end

function get_khi(values1, values2)
    return 10
end

function get_scores(adns, values, equa_diff, func, variable)
    scores = []
    tspan = (values[1][1], values[end][1])
    u0 = values[1][2]
    
    for adn in adns
        f = subs(equa_diff, adn...)
        f = lambdify(f, (func, variable))
        F(u,_,t) = f(u,t)
        ODEProblem(F,u0,tspan)
        try            
            sol = DF.solve(prob)
            adn_score = get_khi(values, sol)
        catch
            adn_score = -Inf
        end
        
        push!(scores, (adn, adn_score))
    end
    
    return scores
end

function create_adns(params_span, n_adns)
    adns = []
    for _ in 1:n_adns
        adn = [(var, rand(span)) for (var, span) in params_span]
        push!(adns, adn)
    end
    return adns
end

function create_childs(adns, n_childs, n_parents)
    childs = []
    
    for _ in n_childs
        parents = sample(adns, n_parents, replace=false)
        child = []
    end
    
    return childs
end

data = CSV.read("dataset/kami.csv")
graph = @df data plot(:x,:y)

#dI (t) = −a_1*I(t) − a_2*I(t)^2 + (a_3 * I(t) + q) * I (t)^(1−(1/n))

@vars I a1 a2 a3 q n

dI = -a1*I - a2*I^2 + (a3 * I + q) * I^(1-(1/n))

params_span = [
    (a1, -10:eps():10),
    (a2, -10:eps():10),
    (a3, -10:eps():10),
    (q, -10:eps():10),
    (n, -10:eps():10)];

values = collect(zip(data[:,:x],data[:,:y]))

find_fit(values, dI, I, t, params_span)


UndefVarError: UndefVarError: adn_score not defined